In [1]:
import numpy as np
import sympy as sp


def from_ham_coeff_symb(N, J_coeffs, h_coeffs, s):
    """
    Construct an MPO from a annealing Hamiltonian for QUBO problems

    Args:
        N: number of qubits
        J_coeffs: coefficients of the long range interactions
        h_coeffs: coefficients of the short range interactions
        s: transverse field strength

    Raises:
        NotImplementedError: Type of Hamiltonian not supported.

    Returns:
        MPO (List[array]): List of tensors representing the MPO (1 tensor for each qubit)
        offset (float): Offset for the energy of the Hamiltonian
    """
    # TODO Simplify MPOs if certain given coefficients are 0

    # Start building the Tensor Network
    i_matrix = np.array([[1, 0], [0, 1]])
    x_matrix = np.array([[0, 1], [1, 0]])
    z_matrix = np.array([[1, 0], [0, -1]])
    
    N_by_2 = int(N / 2)

    a_list = [np.zeros((k + 2, k + 3), dtype=object) for k in range(1, N_by_2)]

    aux = 2 if N % 2 == 0 else 3
    a_list += [np.zeros((N_by_2 + 2, N_by_2 + aux), dtype=object)]

    a_list += [np.zeros((N - k + 3, N - k + 2), dtype=object) for k in range(N_by_2 + 1, N + 1)]

    for k in range(1, N+1):
        a_list[k-1][0,0] = 1
        a_list[k-1][-1,-1] = 1
        a_list[k-1][-2,-1] = sp.Symbol(f'Z_{k-1}')
        a_list[k-1][0,-1] = sp.Symbol(f'(1-{s})')*sp.Symbol(f'X_{k-1}') + sp.Symbol(f'{s}*{h_coeffs[k-1]}')*sp.Symbol(f'Z_{k-1}')
        
        if k < N_by_2:
            a_list[k-1][0,1] = sp.Symbol(f'Z_{k-1}') 
            a_list[k-1][0,k+1] = sp.Symbol(f'{s}*{J_coeffs[k-1,k]}')*sp.Symbol(f'Z_{k-1}')

            for m in range(2, k+1):
                a_list[k-1][m-1, m] = 1
                a_list[k-1][m-1, k+1] = sp.Symbol(f'{s}*{J_coeffs[k-m, k]}')

        elif k == N_by_2:
            for n in range(2, N_by_2 + aux):
                a_list[k-1][0, n-1] = sp.Symbol(f'{s}*{J_coeffs[N_by_2-1, N-n+1]}')*sp.Symbol(f'Z_{k-1}')
                for m in range(2, k+1):
                    a_list[k-1][m-1, n-1] = sp.Symbol(f'{s}*{J_coeffs[N_by_2-m,N-n+1]}')

        else: # k+1 > N_by_2:
            for m in range(2, N-k+2):
                a_list[k-1][0, m-1] = sp.Symbol(f'{s}*{J_coeffs[k-1,N-m+1]}')*sp.Symbol(f'Z_{k-1}')
                a_list[k-1][m-1, m-1] = 1

    a_list[0] = a_list[0][0, :] # other rows are used to propagate information of previous tensors, does not make sense to keep them
    a_list[-1] = a_list[-1][:, -1] # other columns are used to propagate information to the next tensors, does not make sense to keep them

    tn = a_list[0]
    
    for i in range(N - 1):
        tn = np.matmul(tn, a_list[i+1])

    return tn, a_list

In [2]:
import tests.custom_tests as tests

N, values, weights, W_capacity = tests.medium_test()

In [3]:
from exact_solver import Exact_solver


exact_solver = Exact_solver(W_capacity, weights, values)

print('\n \n')
for i in range(len(exact_solver.H_dict)):
    print(f'{i}: {exact_solver.H_dict[i]}')
print('\n \n')

# print(exact_solver.target_ham)
eigenvalues, eigenvectors = np.linalg.eig(exact_solver.target_ham)
gap = sorted(eigenvalues)[:2]
print('gap: ', gap)



[Qibo 0.2.7|INFO|2024-05-22 15:18:58]: Using numpy backend on /CPU:0
[Qibo 0.2.7|WARNING|2024-05-22 15:18:58]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-05-22 15:18:58]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



 

0: {1: -103.9799999999999, 0: -88.38999999999999, 2: -57.50999999999988, 3: -106.31999999999994}
1: {(0, 1): 194.3005, (2, 1): 257.692, (2, 0): 94.081, (3, 1): 363.944, (3, 0): 117.86699999999999, (3, 2): 166.378}
2: 312.9244999999996

 

gap:  [(-585.8505+0j), (-501.4455000000002+0j)]


In [4]:
from ncon import ncon


symb_ham, a_list = from_ham_coeff_symb(N, exact_solver.J_coeffs, exact_solver.h_coeffs, 1)

print(symb_ham)
# print(a_list)

from DMRG.annealing_ham_to_MPO import from_ham_coeff

aux_M = from_ham_coeff(N, exact_solver.J_coeffs, exact_solver.h_coeffs, 1)

aux_M[0] = aux_M[0][0, :] # other rows are used to propagate information of previous tensors, does not make sense to keep them
aux_M[-1] = aux_M[-1][:, -1]

ham = aux_M[0]
for i in range(1, len(aux_M)-1):
    ham = ncon([ham, aux_M[i]], [[1, -2, -4], [1, -1, -3, -5]])
    ham = ham.reshape(ham.shape[0], 2**(i+1), 2**(i+1))
ham = ncon([ham, aux_M[-1]], [[1, -1, -3], [1, -2, -4]])
ham = ham.reshape(2**exact_solver.N, 2**exact_solver.N)

# print(ham)

# perform exact diagonalization
eigenvalues, eigenvectors = np.linalg.eig(ham)
gap = sorted(eigenvalues)[:2]

print('gap: ', gap)

(1-1)*X_0 + (1-1)*X_1 + (1-1)*X_2 + (1-1)*X_3 + 1*-103.9799999999999*Z_1 + 1*-106.31999999999994*Z_3 + 1*-57.50999999999988*Z_2 + 1*-88.38999999999999*Z_0 + 1*194.3005*Z_0*Z_1 + Z_2*(1*257.692*Z_1 + 1*94.081*Z_0) + Z_3*(1*117.86699999999999*Z_0 + 1*166.378*Z_2 + 1*363.944*Z_1)
[[117.867   0.   ]
 [  0.    117.867]]
[[94.081  0.   ]
 [ 0.    94.081]]
gap:  [-585.8504999999999, -501.44550000000015]


In [5]:
for i in range(2**exact_solver.N):
    print(f'Real ham ({i}): ', exact_solver.target_ham[i])
    print(f'cust ham ({i}): ', ham[i])
    print('-----------------')

Real ham (0):  [838.0625+0.j   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j
   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j
   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j
   0.    +0.j]
cust ham (0):  [838.0625   0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.    ]
-----------------
Real ham (1):  [   0.    +0.j -245.6755+0.j    0.    +0.j    0.    +0.j    0.    +0.j
    0.    +0.j    0.    +0.j    0.    +0.j    0.    +0.j    0.    +0.j
    0.    +0.j    0.    +0.j    0.    +0.j    0.    +0.j    0.    +0.j
    0.    +0.j]
cust ham (1):  [   0.     -245.6755    0.        0.        0.        0.        0.
    0.        0.        0.        0.        0.        0.        0.
    0.        0.    ]
-----------------
Real ham (2):  [  0.    +0.j   0.    +0.j -83.2195+0.j   0.    +0.j   0.    +0.j
   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j   0.    +0.j
   0.    +0.j   